# cw_cnnv2

# 📦 Set-up 

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F


In [2]:
import os
import re
import sys
import numpy as np
import pandas as pd
import torch
import time
from google.colab import drive
from skimage import io, transform
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from torchvision import transforms, utils
from skimage import io, transform
from torch.utils.data.sampler import Sampler, WeightedRandomSampler
from sklearn import svm, metrics

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [3]:
drive.mount('/content/drive')

# TODO: Fill in the Google Drive path where you uploaded the lab materials
# Example: GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'Colab Notebooks/Lab materials 01-20210104'

GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = '/content/drive/MyDrive/Computer Vision Coursework/CW_Folder_PG' 
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))

sys.path.append('Coursework/CW_Folder_PG/Code/')

Mounted at /content/drive
['.DS_Store', 'Models', 'CW_Dataset', 'Code', 'Video', 'test_functions_scratch.ipynb']


In [4]:
# Identify path to zipped dataset
zip_path = os.path.join(GOOGLE_DRIVE_PATH, 'CW_Dataset/CW_Dataset.zip')

# Copy it to Colab
!cp '{zip_path}' .

# Unzip it
!yes|unzip -q CW_Dataset.zip

# Delete zipped version from Colab (not from Drive)
!rm CW_Dataset.zip

In [5]:
def loadimages(root_dir, label_path):
    """Return an array of images and array of labels.

        Args:
            root_dir (string): Folder containing all the images.
            label_path (string): Path to text file containing image filename and labels in this order.
    """
    #import filenames and respective labels
    df = pd.read_csv(label_path,
                    delimiter=' ',
                    header=None,
                    names=['filename', 'label'])
    df.sort_values('filename', inplace=True)
    #import images
    images = []
    #!sort image folder
    image_folder = sorted(os.listdir(root_dir))
    #generator to extract each image path
    images_in_folder = (file for file in image_folder if file.endswith('.jpg'))
    count = 0 #index
    for imagepath, imagepath_label in zip(images_in_folder, (df['filename'])):
        count += 1 #check if filenames and filenames in labels list are equal
        if re.findall('\d+', imagepath) == re.findall('\d+', imagepath_label):
            image = io.imread(os.path.join(root_dir, imagepath))
            images.append(image)
        else:
            print(f'Found unmatched image file {imagepath} and label {imagepath_label} \
                at index {count}')
            pass

    return np.array(images), np.array(df['label'])

In [6]:
X, y = loadimages(
    '/content/train',
    '/content/labels/list_label_train.txt'
)

In [7]:
test, test_y = loadimages(
    '/content/test',
    '/content/labels/list_label_test.txt'
)

In [8]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, shuffle=True, stratify=y)

In [9]:
class DatasetTorch(Dataset):
    #adapted from https://pytorch.org/tutorials/beginner/data_loading_tutorial.html
    """Images dataset for pytorch."""

    def __init__(self, root_dir, label_path, transform=None):
        """
        Args:
            csv_file (string): Path to the text file with *exact image filenames* and  respective labels.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        #self.labels = pd.read_csv(
                                  #label_path,
                                  #delimiter=' ',
                                  #header=None,
                                  #names=['filename', 'label']
                                #)
        #self.root_dir = root_dir
        self.labels = label_path - 1
        self.root_dir = root_dir
        self.transform = transform
        #self.image_path = sorted(os.listdir(root_dir))
        #self.transform = transforms.Compose(
                                #[transforms.Normalize(),
                                #transforms.ToTensor()]
                                #)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        #if torch.is_tensor(idx):
            #idx = idx.tolist()

        #img_name = os.path.join(self.root_dir,
                                #self.labels.iloc[idx, 0])
        #image = io.imread(img_name)
        image = self.root_dir[idx]
        #label = self.labels.iloc[idx, 1]
        label = self.labels[idx]
        

        if self.transform:
            image = self.transform(image)
        
        return image, label

In [10]:
dataset_images = DatasetTorch(X_train,
                   y_train,
                   transform=transforms.Compose([
                    transforms.ToTensor(),                             
                    transforms.Normalize([0.5752, 0.4495, 0.4012], [0.2654, 0.2422, 0.2407])                             
                   ]
                       
                   ))

dataloader = DataLoader(dataset_images, batch_size=32,
                        shuffle=True, num_workers=0)

dataset_images_val = DatasetTorch(X_val,
                   y_val,
                   transform=transforms.Compose([
                    transforms.ToTensor(),                             
                    transforms.Normalize([0.5752, 0.4495, 0.4012], [0.2654, 0.2422, 0.2407])                             
                   ]
                       
                   ))


test_loader = DataLoader(dataset_images_val, batch_size=32,
                        shuffle=True, num_workers=0)




## ⚛ Seting up the Convulution models

### 1️⃣ Simple

In [12]:

class ConvNet1(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(16*22*22, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 7)
        # Define proportion or neurons to dropout
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16*22*22)
        x = self.dropout(x)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

### 2️⃣ Larger one

In [ ]:
class ConvNet2(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet2, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(
                    in_channels=3,
                    out_channels=32,
                    kernel_size=1,
                    stride=1,
                    padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2))

        self.layer2 = nn.Sequential(
            nn.Conv2d(
                    in_channels=32,
                    out_channels=64,
                    kernel_size=1,
                    stride=1,
                    padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Dropout(p=0.5))

        self.layer3 = nn.Sequential(
            nn.Conv2d(
                    in_channels=64,
                    out_channels=128,
                    kernel_size=1,
                    stride=1,
                    padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2))

        self.layer4 = nn.Sequential(
            nn.Conv2d(
                    in_channels=128,
                    out_channels=256,
                    kernel_size=1,
                    stride=1,
                    padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Dropout(p=0.5))
        
        self.layer5 = nn.Sequential(
            nn.Conv2d(
                    in_channels=256,
                    out_channels=512,
                    kernel_size=1,
                    stride=1,
                    padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2))

        self.fc= nn.Sequential(
            nn.Linear(512*5*5, 4096),
            nn.ReLU(),
            nn.Linear(4096, 1028),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(1028, 256),
            nn.Linear(256, 7)
                                )
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

## 🏃 Training Loop

In [ ]:
net.eval()

predictions = []
targets = []

# Run the model on some test examples
with torch.no_grad():
    correct, total, cumu_loss = 0, 0, 0
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        # Get logits
        outputs = net(images)
        # Calculate loss
        loss = criterion(outputs, labels)
        cumu_loss += loss.item()
        #Get predictions
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # Save predictions
        pred = predicted.detach().cpu().detach().numpy()
        labels_list = labels.detach().cpu().detach().numpy()

        #Flatten
        for i in range(len(pred)):
          predictions.append(pred[i])
          targets.append(labels_list[i])
            

predictions, targets


In [13]:
# Define evaluation function

def test(model, test_loader):
    model.eval()

    predictions = []
    targets = []
    
    # Run the model on some test examples
    with torch.no_grad():
        correct, total, cumu_loss = 0, 0, 0
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            # Get logits
            outputs = model(images)
            # Calculate loss
            loss = criterion(outputs, labels)
            cumu_loss += loss.item()
            #Get predictions
            _, predicted = torch.max(outputs.data, 1)
            #total += labels.size(0)
            #correct += (predicted == labels).sum().item()

            # Save predictions
            pred = predicted.detach().cpu().detach().numpy()
            labels_list = labels.detach().cpu().detach().numpy()

            #Flatten
            for i in range(len(pred)):
              predictions.append(pred[i])
              targets.append(labels_list[i])
            

        return predictions, targets


In [16]:
min_val_loss = 100_000
patience = 10
max_balance = -1
accuracy_score_epoch = []


# Initialize model
net = ConvNet1()

# transfer network to GPU
net.to(device)

# define the loss and the optimizer
criterion = nn.CrossEntropyLoss(weight=torch.from_numpy(weight).float().to(device))
optimizer = optim.Adam(net.parameters(), lr=0.001)

t0 = time.time()

for epoch in range(100):

    running_loss = 0.0
    for i, data in enumerate(dataloader, 0):
        # transfer data to GPU
        inputs, labels = data[0].to(device), data[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward pass ->
        outputs = net(inputs)
        loss = criterion(outputs, labels)

        # backward pass <-
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs, 1)
        #print(metrics.classification_report(labels.cpu(), predicted.cpu()))
        # print statistics (loss.item() returns the mean loss in the mini-batch)
        running_loss += loss.item()
    #print('[%d, %5d] loss: %.3f' %
    #(epoch + 1, i + 1, running_loss / 2000))
    running_loss = 0.0

    predictions, targets = test(net, test_loader)
    balance_accuracy = metrics.balanced_accuracy_score(targets,
                                                      predictions)
    accuracy_score_epoch.append(balance_accuracy)                                                                 #val_loss += loss                                                                #val_loss = val_loss / len(trainloader)
    if balance_accuracy > max_balance:
    #Saving the model
      print(f'The Balanced accuracy is {balance_accuracy}')
      max_balance = balance_accuracy
      best_model = torch.save(net.state_dict(), 'CNN_model.pt')
      #print('Min loss %0.2f' % min_val_loss)
      f1_score = metrics.f1_score(targets,
                                  predictions,
                                  average=None)
      trigger_times = 0
      #min_val_loss = val_loss
    else: 
      trigger_times += 1
      print(f'The Balanced accuracy is {balance_accuracy}')
    if trigger_times >= patience:
        print('Early stopping!\nStart to test process.')
        break

print('Finished Training on GPU: total time in seconds =', time.time() - t0)

The Balanced accuracy is 0.48109488353732016
The Balanced accuracy is 0.5565457468398815
The Balanced accuracy is 0.5670918252918116
The Balanced accuracy is 0.6236265620023332
The Balanced accuracy is 0.594061670405828
The Balanced accuracy is 0.5899807335835471
The Balanced accuracy is 0.569025882148384
The Balanced accuracy is 0.6079482457921598
The Balanced accuracy is 0.5809682006476973
The Balanced accuracy is 0.616154299595428
The Balanced accuracy is 0.600399621678405
The Balanced accuracy is 0.6009881460305065
The Balanced accuracy is 0.593705943034246
The Balanced accuracy is 0.598035566309585
Early stopping!
Start to test process.
Finished Training on GPU: total time in seconds = 49.456464767456055


1. The Simpler CONV has an accuracy 72.62729124236253% and the Larger one ~48.14663951120163% with a balanced accuracy of 0.5995134407520585% and 0.2221346742004043 respectively. However the smaller Conv has jsut one dropout layer which may make it proe to overfitting. On the other hand deeped Neural networks are prone to overfitting as well.

I'll create another one which is not that deep.




2. Let's try the weighted random sampler. 

For the larger Conv : The weighted random sampler increased the accuracy to 32% and 25% balanced accuracy. That accuracy lags behind behind the first one. Suggests the model is not relialing that much on the majority class

For the Simple one: Did pretty much the same with 67.12830957230143% accuracy and The Balanced accuracy is 0.5778150063329726

3. Next I'll try the class weights inside the loss function:
  For the smaller got an accuracy  of the model at 67.25050916496944% and;
  The Balanced accuracy is 0.583481414080001

  For the larger 38.90020366598778%
The Balanced accuracy is 0.14285714285714285

The class weights did not lead to that much a decrease in the average accuracy, at least for the smaller convulution network. I conjecture using class weights is better even though the total accuracy has decreased as it will probably have better generalization.

In [15]:
# Adapted from https://discuss.pytorch.org/t/how-to-handle-imbalanced-classes/11264/5
class_sample_count = np.unique(y_train, return_counts=True)[1]
y = y_train - 1
weight = 1 / class_sample_count
samples_weight = weight[y]

samples_weight = torch.from_numpy(samples_weight)
samples_weigth = samples_weight.double()
sampler = WeightedRandomSampler(samples_weight, len(samples_weight), replacement=True)

In [ ]:
weighted_sampler = WeightedRandomSampler(
    weights=samples_weights,
    num_samples=len(y_train),
    replacement=True
)

In [ ]:
dataset_images = DatasetTorch(X_train,
                   y_train,
                   transform=transforms.Compose([
                    transforms.ToTensor(),                             
                    transforms.Normalize([0.5752, 0.4495, 0.4012], [0.2654, 0.2422, 0.2407]),                              
                   ]
                       
                   ))

dataloader = DataLoader(dataset_images, batch_size=32,
                         num_workers=1, sampler=sampler,pin_memory=True)

In [ ]:
dataset_images_val = DatasetTorch(test,
                   test_y,
                   transform=transforms.Compose([
                    transforms.ToTensor(),                             
                    transforms.Normalize([0.5752, 0.4495, 0.4012], [0.2654, 0.2422, 0.2407])                             
                   ]
                       
                   ))

test_loader = DataLoader(dataset_images_val, batch_size=32,
                        shuffle=True, num_workers=0)


In [ ]:
net.eval()

predictions = []
targets = []

# Run the model on some test examples
with torch.no_grad():
    correct, total, cumu_loss = 0, 0, 0
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        # Get logits
        outputs = net(images)
        # Calculate loss
        loss = criterion(outputs, labels)
        cumu_loss += loss.item()
        #Get predictions
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # Save predictions
        pred = predicted.detach().cpu().detach().numpy()
        labels_list = labels.detach().cpu().detach().numpy()

        #Flatten
        for i in range(len(pred)):
          predictions.append(pred[i])
          targets.append(labels_list[i])

    print(f"Accuracy of the model on the {total} " +
          f"test images: {100 * correct / total}%")
    #Get balanced accruacy
    #Get F_score
    #if logging:
        #wandb.log({"test_accuracy": correct / total})

# Save the model in the exchangeable ONNX format 
#if logging:
  #torch.onnx.export(model, images, "model.onnx")
  #wandb.save("model.onnx")


Accuracy of the model on the 3068 test images: 71.77314211212516%


In [ ]:
f1_score

array([0.99040307, 0.99099099, 0.97183099, 0.99790576, 0.98997494,
       0.99646643, 0.98703888])

In [ ]:
Finished Training on GPU: total time in seconds = 102.6555826663971
